In [1]:
from typing_extensions import TypedDict, Annotated, List, Literal
from langchain_core.messages import AnyMessage, AIMessage, HumanMessage, SystemMessage
from langgraph.graph import add_messages
from typing import Dict, List, Optional, Union, Literal
from pydantic import BaseModel, Field
from IPython.display import Image, display 
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages

In [2]:
## documents
class DocumentSection(BaseModel):
    title: str = Field(description="The title of the section")
    content: str = Field(description="The content of the section")
    

## security review
class SecurityReview(TypedDict):
    sec_id : str
    review : str
    file_path : str
    recommendation : str
    priority : Literal["high", "medium", "low"]
    
## test cases
class TestCase(TypedDict):
    test_id : str 
    description : str 
    steps : List[str]
    status : Literal["draft", "pass", "fail"] = "draft"

In [3]:
## Project Requirements
class ProjectRequirements(BaseModel):
    title: str = Field(description="The title of the project")
    description: str = Field(description="The description of the project")
    requirements : List[str] = Field(description="The requirements of the project")

In [4]:
## User story
class UserStory(BaseModel):
    story_id : str = Field(description="The id of the user story")
    title : str = Field(description="The title of the user story")
    description : str = Field(description="The description of the user story")
    acceptance_criteria : List[str] = Field(description="The acceptance criteria of the user story")

In [5]:
class SDLCState(BaseModel):
    project_requirements : ProjectRequirements
    revised_count : int = Field(default=0, description="The number of times the revised")
    # User story
    user_stories : List[UserStory] = []
    user_story_messages : Annotated[list, add_messages] = []
    user_story_status : Literal["pending", "in_progress", "pending_approval", "feedback", "approved"] = "pending"
    
    # functional documents
    functional_documents : List[DocumentSection] = []
    functional_messages : Annotated[list, add_messages] = []
    functional_status: Literal["pending", "in_progress", "pending_approval", "feedback", "approved"] = "pending"
    
    # technical documents
    technical_documents : List[DocumentSection] = []
    technical_messages : Annotated[list, add_messages] = []
    technical_status: Literal["pending", "in_progress", "pending_approval", "feedback", "approved"] = "pending"
    
    # frontend code
    frontend_code : str = Field(default='', description="The frontend code")
    frontend_messages: Annotated[list, add_messages] = []
    frontend_status: Literal["pending", "in_progress", "pending_approval", "feedback", "approved"] = "pending"
    
    # backend code
    backend_code : str = Field(default= '', description="The backend code")
    backend_messages: Annotated[list, add_messages] = []
    backend_status: Literal["pending", "in_progress", "pending_approval", "feedback", "approved"] = "pending"

    # security review
    security_reviews : List[SecurityReview] = []
    security_reviews_messages: Annotated[list, add_messages] = []
    security_reviews_status : Literal["pending", "in_progress", "pending_approval", "feedback", "approved"] = "pending"

    ## test cases
    test_cases : List[TestCase] = []
    test_cases_messages: Annotated[list, add_messages] = []
    test_cases_status : Literal["pending", "in_progress", "pending_approval", "feedback", "approved"] = "pending"

    ## qa testing
    qa_testing : List[TestCase] = []
    qa_testing_messages: Annotated[list, add_messages] = []
    qa_testing_status : Literal["pending", "in_progress", "pending_approval", "feedback", "approved"] = "pending"

    ## Code deployment
    deployment : str = Field(default='', description="The code deployment")
    deployment_messages: Annotated[list, add_messages] = []


## Helper functions

In [6]:
def generate_user_stories_from_llm(project_title, project_description, requirements):
    user_stories = [{'story_id': 'US-001',
        'title': 'Secure Account with Multi-Factor Authentication',
        'description': 'As a user, I want to secure my account with multi-factor authentication so that my transactions are protected.',
        'acceptance_criteria': ['User can enable biometric authentication (fingerprint and facial recognition).',
        'User can set up MPIN for additional security.',
        'User can enable two-factor authentication for transactions.',
        'System alerts user of authentication failures.']}]
    
    return user_stories

def revised_user_stories_from_llm(user_stories, user_review):
    revised_user_stories = [{'story_id': 'US-001',
        'title': 'Secure Account with Multi-Factor Authentication',
        'description': 'As a user, I want to secure my account with multi-factor authentication so that my transactions are protected.',
        'acceptance_criteria': ['User can enable biometric authentication (fingerprint and facial recognition).',
        'User can set up MPIN for additional security.',
        'User can enable two-factor authentication for transactions.',
        'System alerts user of authentication failures.']},
        {'story_id': 'US-002',
        'title': 'Link Multiple Bank Accounts',
        'description': 'As a user, I want to link multiple bank accounts so that I can transfer funds instantly.',
        'acceptance_criteria': ['User can add multiple bank accounts to their profile.',
        'User can select any linked account for fund transfers.',
        'Linked accounts display current balance.',
        'System confirms successful linking of accounts.']}]
    return revised_user_stories

### User stories

In [7]:
## User stories 
def process_project_requirements(state : SDLCState) -> SDLCState:
    print("In process_project_requirements...")
    return {
        "user_story_messages" : AIMessage(content="I've received your project requirements. I'll now generate user stories based on these requirements.")
    }
    
def generate_user_stories(state : SDLCState) -> SDLCState:
    print("In generate_user_stories...") 
    project_title = state.project_requirements.title
    project_description = state.project_requirements.description
    requirements = state.project_requirements.requirements
    user_stories = generate_user_stories_from_llm(project_title, project_description, requirements);
    
    return {
        "user_story_status" : 'pending_approval',
        "user_stories" : user_stories,
        "user_story_messages" : AIMessage(
            content = f"Based on your requirements, I've generated {len(user_stories)} user stories. Please review these user stories and provide feedback or type 'Approved' if you're satisfied."
        ),
        "revised_count" : 0
    }

def review_user_stories(state : SDLCState) -> SDLCState:
    print("In review_user_stories...")
    user_review = state.user_story_messages[-1].content
    user_review = user_review.lower().strip()
    print("user_review : ", user_review)  
    if user_review == "approved":
        return {
            "user_story_messages" : AIMessage(content="Great! Your user stories have been finalized. You can now proceed with your design process."),
            "user_story_status" : "approved",
        }
    else:
        return {
            "user_story_messages" :  AIMessage(content="I've received your feedback. I'll revise the user stories accordingly."),
            "user_story_status" : "feedback"
        }

def should_revise_user_stories(state : SDLCState) -> Literal["feedback", "approved"]:
    return "approved" if state.user_story_status == "approved" else "feedback"   

def revised_user_stories(state : SDLCState) -> SDLCState:
    print("In revised_user_stories...")
    user_review = state.user_story_messages[-2].content
    user_review = user_review.lower().strip()
    if state.user_story_status == 'feedback':
        revised_count = state.revised_count + 1
        print("revised_count : ", revised_count)
        if revised_count == 3:
            return {
                "user_story_messages" :  AIMessage(
                    content = f"User stories have been revision maxed out. Please review the user stories and continue with the next step."
                ),
                "user_story_status" : "approved"
            }
        user_stories = revised_user_stories_from_llm(state.user_stories, user_review)
        return {
                "user_stories" : user_stories,
                "user_story_messages" : AIMessage(
                    content = "I've revised the user stories based on your feedback.\n\nPlease review these updated user stories and provide additional feedback or type 'Approved' if you're satisfied."),
                "user_story_status" : "pending_approval",
                "revised_count" : revised_count
            }

### Functional documents

In [8]:
def generate_functional_documents_from_llm(user_stories, project_requirements):
    document = DocumentSection(
        title = f"FUNCTIONAL REQUIREMENTS",
        content = "FR-1: The system shall provide a Forgot Password option on the login page.")
    functional_documents = [document]
    return functional_documents


def revised_functional_documents_from_llm(docs, user_feedback):
    document = DocumentSection(
        title = f"Revised FUNCTIONAL REQUIREMENTS",
        content = "FR-1: The system shall provide a Forgot Password option on the login page.")
    functional_documents = [document]
    return functional_documents

def generate_technical_documents_from_llm(user_stories, project_requirements):
    sections=[
        DocumentSection(
            title="Technical Document: MODULES & COMPONENTS DESIGN",
            content="Auth API Module: Exposes RESTful endpoints for password"
        )
    ]
    return sections

def revised_technical_documents_from_llm(docs, user_feedback):    
    sections=[
        DocumentSection(
            title="Revised Technical Document: MODULES & COMPONENTS DESIGN",
            content="Auth API Module: Exposes RESTful endpoints for password"
        )
    ]
    return sections

In [9]:
def create_functional_documents(state : SDLCState) -> SDLCState:
    print("In create_functional_documents...")
    doc_type = "functional"
    user_stories = state.user_stories
    project_requirements = state.project_requirements
    documents = generate_functional_documents_from_llm(user_stories, project_requirements)
    return {
        f"{doc_type}_documents": documents,
        f"{doc_type}_status": 'pending_approval',
        f"{doc_type}_messages": AIMessage(
            content=f"Please review {doc_type} design document and provide feedback or type 'Approved' if you're satisfied."
        ),
    }

def review_functional_documents(state : SDLCState) -> SDLCState:
    print("In review_functional_documents...")
    doc_type = "functional"
    user_feedback = state.functional_messages[-1].content.lower().strip()
    print("user feedback:", user_feedback)
    approved = (user_feedback == "approved")
    response = {
        f"{doc_type}_messages": AIMessage(
            content="Great! Your design documents have been finalized. You can now proceed with your design process."
            if approved else
            "I've received your feedback. I'll revise the design documents accordingly."
        ),
        f"{doc_type}_status": "approved" if approved else "feedback",
    }
    return response

def should_revise_function_documents(state : SDLCState) -> Literal["feedback", "approved"]:
    return "approved" if state.functional_status == "approved" else "feedback"  

def revise_functional_documents(state : SDLCState) -> SDLCState:
    print("In revise_functional_documents...")
    doc_type = "functional"
    user_feedback = state.functional_messages[-2].content.lower().strip()
    print("user feedback:", user_feedback)
    revised_count = state.revised_count + 1
    print("revised_count :", revised_count)

    if revised_count == 3:
        return {
            f"{doc_type}_messages": AIMessage(
                content="Design documents have been revision maxed out. Please review the these documents and continue with the next step."
            ),
            f"{doc_type}_status": "approved",
        }

    documents = revised_functional_documents_from_llm(state.functional_documents, user_feedback)
    return {
        f"{doc_type}_documents": documents,
        f"{doc_type}_messages": AIMessage(
            content=f"Please review revised {doc_type} design documents and provide additional feedback or type 'Approved' if you're satisfied."
        ),
        f"{doc_type}_status": "pending_approval",
        "revised_count": revised_count
    }


### Technical documents

In [10]:
def create_technical_documents(state : SDLCState) -> SDLCState:
    print("In create_technical_documents...")
    doc_type = "technical"
    user_stories = state.user_stories
    project_requirements = state.project_requirements
    documents = generate_technical_documents_from_llm(user_stories, project_requirements)
    return {
        f"{doc_type}_documents": documents,
        f"{doc_type}_status": 'pending_approval',
        f"{doc_type}_messages": AIMessage(
            content=f"Please review {doc_type} design document and provide feedback or type 'Approved' if you're satisfied."
        ),
    }

def review_technical_documents(state : SDLCState) -> SDLCState:
    print("In review_technical_documents...")
    doc_type = "technical"
    user_feedback = state.technical_messages[-1].content.lower().strip()
    print("user feedback:", user_feedback)
    approved = (user_feedback == "approved")
    response = {
        f"{doc_type}_messages": AIMessage(
            content="Great! Your design documents have been finalized. You can now proceed with your design process."
            if approved else
            "I've received your feedback. I'll revise the design documents accordingly."
        ),
        f"{doc_type}_status": "approved" if approved else "feedback",
    }
    return response

def should_revise_technical_documents(state : SDLCState) -> Literal["feedback", "approved"]:
    return "approved" if state.technical_status == "approved" else "feedback" 

def revise_technical_documents(state : SDLCState) -> SDLCState:
    print("In revise_technical_documents...")
    doc_type = "technical"
    user_feedback = state.technical_messages[-2].content.lower().strip()
    print("user feedback:", user_feedback)
    revised_count = state.revised_count + 1
    print("revised_count :", revised_count)

    if revised_count == 3:
        return {
            f"{doc_type}_messages": AIMessage(
                content="Design documents have been revision maxed out. Please review the these documents and continue with the next step."
            ),
            f"{doc_type}_status": "approved",
        }

    documents = revised_technical_documents_from_llm(state.technical_documents, user_feedback)
    return {
        f"{doc_type}_documents": documents,
        f"{doc_type}_messages": AIMessage(
            content=f"Please review revised {doc_type} design documents and provide additional feedback or type 'Approved' if you're satisfied."
        ),
        f"{doc_type}_status": "pending_approval",
        "revised_count": revised_count
    }

## Frontend code

In [11]:
def generate_frontend_code_from_llm(project_requirements, user_stories):
    frontend_code = "console.log('frontend code')"
    return frontend_code 

def revised_frontend_code_from_llm(code, user_feedback):
    frontend_code = "console.log('Revised frontend code')"
    return frontend_code

In [12]:
def generate_frontend_code(state : SDLCState) -> SDLCState:
    print("In generate_frontend_code...")
    generate_code = generate_frontend_code_from_llm(state.project_requirements, state.user_stories)
    code_type = "frontend"

    return {
        f"{code_type}_code" : generate_code,
        f"{code_type}_status": 'pending_approval',
        f"{code_type}_messages": AIMessage(
            content=f"Please review {code_type} design document and provide feedback or type 'Approved' if you're satisfied."
        ),
    }

def review_frontend_code(state : SDLCState) -> SDLCState:
    print("In review_frontend_code")
    user_feedback = state.frontend_messages[-1].content.lower().strip()
    print("user feedback:", user_feedback)
    approved = user_feedback == "approved"
    code_type = "frontend"
    return {
        f"{code_type}_messages":AIMessage(
            content="Great! Code have been finalized. You can now proceed with next steps."
            if approved else
            "I've received your feedback. I'll revise the design documents accordingly."),
        f"{code_type}_status": "approved" if approved else "feedback"
    }

def should_fix_frontend_code(state : SDLCState) -> Literal["feedback", "approved"]:
    return "approved" if state.frontend_status == 'approved' else 'feedback'

def fix_frontend_code(state : SDLCState) -> SDLCState:
    print("In fix_frontend_code...")
    code_type = "frontend"
    user_feedback = state.frontend_messages[-2].content.lower().strip()
    revised_count = state.revised_count + 1
    print("revised_count :", revised_count)

    if revised_count == 3:
        return {
            f"{code_type}_messages": AIMessage(
                content="Code have been revision maxed out. Please review the code and continue with the next step."
            ),
            f"{code_type}_status": "approved"
        }
    
    revised_code = revised_frontend_code_from_llm(state.frontend_code, user_feedback)
    return {
        f"{code_type}_code": revised_code,
        f"{code_type}_messages": AIMessage(
            content=f"Please review revised {code_type} code and provide additional feedback or type 'Approved' if you're satisfied."
        ),
        f"{code_type}_status": "pending_approval",
        "revised_count": revised_count
    }

## Backend Code

In [13]:
def generate_backend_code_from_llm(project_requirements, user_stories):
    backend_code = "print('backend code')"
    return backend_code 

def revised_backend_code_from_llm(code, user_feedback):
    backend_code = "print('Revised backend code')"
    return backend_code

In [14]:
def generate_backend_code(state : SDLCState) -> SDLCState:
    print("In generate_backend_code...")
    generate_code = generate_backend_code_from_llm(state.project_requirements, state.user_stories)
    code_type = "backend"

    return {
        f"{code_type}_code" : generate_code,
        f"{code_type}_status": 'pending_approval',
        f"{code_type}_messages": AIMessage(
            content=f"Please review {code_type} design document and provide feedback or type 'Approved' if you're satisfied."
        ),
    }

def review_backend_code(state : SDLCState) -> SDLCState:
    print("In review_backend_code")
    user_feedback = state.backend_messages[-1].content.lower().strip()
    print("user feedback:", user_feedback)
    approved = user_feedback == "approved"
    code_type = "backend"
    return {
        f"{code_type}_messages":AIMessage(
            content="Great! Code have been finalized. You can now proceed with next steps."
            if approved else
            "I've received your feedback. I'll revise the design documents accordingly."),
        f"{code_type}_status": "approved" if approved else "feedback"
    }

def should_fix_backend_code(state : SDLCState) -> Literal["feedback", "approved"]:
    return "approved" if state.backend_status == 'approved' else 'feedback'

def fix_backend_code(state : SDLCState) -> SDLCState:
    print("In fix_backend_code...")
    code_type = "backend"
    user_feedback = state.backend_messages[-2].content.lower().strip()
    revised_count = state.revised_count + 1
    print("revised_count :", revised_count)

    if revised_count == 3:
        return {
            f"{code_type}_messages": AIMessage(
                content="Code have been revision maxed out. Please review the code and continue with the next step."
            ),
            f"{code_type}_status": "approved"
        }
    
    revised_code = revised_backend_code_from_llm(state.backend_code, user_feedback)
    return {
        f"{code_type}_code": revised_code,
        f"{code_type}_messages": AIMessage(
            content=f"Please review revised {code_type} code and provide additional feedback or type 'Approved' if you're satisfied."
        ),
        f"{code_type}_status": "pending_approval",
        "revised_count": revised_count
    }

## Security Review

In [15]:
def generate_security_reviews_from_llm(code):
    security_reviews = [SecurityReview(
        sec_id="SC-001", 
        review="security review", 
        file_path="backend_code.py", 
        recommendation="fix",
        priority="high")]
    return security_reviews 


def revised_backend_code_with_security_reviews_from_llm(code, reviews, user_feedback):
    backend_code = "print('Revised backend code with security reviews')"
    return backend_code



In [16]:
def check_code_for_security_issues(state : SDLCState) -> SDLCState:
    print("In check_code_for_security_issues...")
    security_reviews = generate_security_reviews_from_llm(state.backend_code)
    return {
        "security_reviews": security_reviews,
        "security_reviews_status": "pending_approval",
        "security_reviews_messages": AIMessage(
            content=f"Please review security reviews and provide feedback or type 'Approved' if you're satisfied."
        ),
    }

def security_review(state : SDLCState) -> SDLCState:
    print("In security_review...")
    user_feedback = state.security_reviews_messages[-1].content.lower().strip()
    print("user feedback:", user_feedback)
    approved = user_feedback == "approved"
    return {
        "security_reviews_status": "approved" if approved else "feedback",
        "security_reviews_messages": AIMessage(
            content="Great! Security reviews have been finalized. You can now proceed with next steps."
            if approved else
            "I've received your feedback. I'll revise the security reviews accordingly."),
    }

def should_fix_code_after_security_review(state : SDLCState) -> Literal["feedback", "approved"]:
    return "approved" if state.security_reviews_status == 'approved' else 'feedback'

def fix_code_after_security_review(state : SDLCState) -> SDLCState:
    print("In fix_code_after_security_review...")
    code_type = "backend"
    user_feedback = state.security_reviews_messages[-2].content.lower().strip()
    revised_count = state.revised_count + 1
    print("revised_count :", revised_count)

    if revised_count == 3:
        return {
            f"{code_type}_messages": AIMessage(
                content="Code have been revision maxed out. Please review the code and continue with the next step."
            ),
            f"{code_type}_status": "approved"
        }
    
    revised_code = revised_backend_code_with_security_reviews_from_llm(state.backend_code, state.security_reviews, user_feedback)
    return {
        f"{code_type}_code": revised_code,
        f"{code_type}_messages": AIMessage(
            content=f"Please review revised {code_type} code and provide additional feedback or type 'Approved' if you're satisfied."
        ),
        f"{code_type}_status": "pending_approval",
        "revised_count": revised_count
    }

## Test Cases

In [17]:
def generate_test_cases_from_llm(project_requirements, user_stories):
    test_cases = [
        TestCase(
        test_id="TC-001", 
        description="test_case_1", 
        steps=["step1", "step2", "step3"],
        status="pending")]
    return test_cases

def revised_test_cases_from_llm(test_cases, user_feedback):
    test_cases = [
        TestCase(
        test_id="TC-001", 
        description="Revised test_case_2", 
        steps=["step1", "step2", "step3"],
        status="pending")
    ]
    return test_cases


In [18]:
def write_test_cases(state : SDLCState) -> SDLCState: 
    print("In write_test_cases...")
    test_cases = generate_test_cases_from_llm(state.project_requirements, state.user_stories)
    return {
        "test_cases": test_cases,
        "test_cases_status": "pending_approval",
        "test_cases_messages": AIMessage(
            content=f"Please review test cases and provide feedback or type 'Approved' if you're satisfied."
        ),
    }


def test_cases_review(state : SDLCState) -> SDLCState:
    print("In test_cases_review...")
    user_feedback = state.test_cases_messages[-1].content.lower().strip()
    print("user feedback:", user_feedback)
    approved = user_feedback == "approved"
    return {
        "test_cases_status": "approved" if approved else "feedback",
        "test_cases_messages": AIMessage(
            content="Great! Test cases have been finalized. You can now proceed with next steps."
            if approved else
            "I've received your feedback. I'll revise the test cases accordingly."),
    }

def should_fix_test_cases(state : SDLCState) -> Literal["feedback", "approved"]:
    return "approved" if state.test_cases_status == 'approved' else 'feedback'

def fixed_testcases_after_review(state : SDLCState) -> SDLCState:
    print("In fixed_testcases_after_review...")
    user_feedback = state.test_cases_messages[-2].content.lower().strip()
    revised_count = state.revised_count + 1
    print("revised_count :", revised_count)

    if revised_count == 3:
        return {
            "test_cases_messages": AIMessage(
                content="Test cases have been revision maxed out. Please review the test cases and continue with the next step."
            ),
            "test_cases_status": "approved"
        }
    
    revised_test_cases = revised_test_cases_from_llm(state.test_cases, user_feedback)
    return {
        "test_cases": revised_test_cases,
        "test_cases_messages": AIMessage(
            content=f"Please review revised test cases and provide additional feedback or type 'Approved' if you're satisfied."
        ),
        "test_cases_status": "pending_approval",
        "revised_count": revised_count
    }

## QA testing

In [19]:
test_cases = [
        TestCase(
        test_id="TC-001", 
        description="test_case_1", 
        steps=["step1", "step2", "step3"],
        status="pending")]

In [20]:
def qa_testing_from_llm(test_cases):
    qa_testing = [
        TestCase(
            test_id="TC-001", 
            description="test_case_1", 
            steps=["step1", "step2", "step3"],
            status="passed"
        ),
    ]
    return qa_testing  # Returns a list of TestCase objects


def revised_backend_code_with_qa_testing_from_llm(code, failed_test_cases):
    backend_code = "print('Revised backend code with qa testing')"
    return backend_code

In [21]:
def perform_qa_testing(state : SDLCState) -> SDLCState:
    print("In perform_qa_testing...")
    test_cases = state.test_cases
    qa_testing = qa_testing_from_llm(test_cases)
    failed_test_cases = [test_case for test_case in qa_testing if test_case['status'] == "failed"]
    passed = len(failed_test_cases) == 0
    return {
        "qa_testing": qa_testing,
        "qa_testing_status": "passed" if passed else "failed",
        "qa_testing_messages": AIMessage(
            content="Great! QA testing have been finalized. You can now proceed with next steps."
            if passed else
            "I'll revise the code accordingly."),
    }

def should_fix_after_qa_testing(state : SDLCState) -> Literal["passed", "failed"]:
    return "passed" if state.qa_testing_status == 'passed' else 'failed'

def fixed_code_after_qa_testing(state : SDLCState) -> SDLCState:
    print("In fixed_code_after_qa_testing...")
    code_type = "backend"
    revised_count = state.revised_count + 1
    print("revised_count :", revised_count)

    if revised_count == 3:
        return {
            f"{code_type}_messages": AIMessage(
                content="Code have been revision maxed out. Please review the code and continue with the next step."
            ),
            f"{code_type}_status": "approved"
        }
    
    failed_test_cases = [test_case for test_case in state.qa_testing if test_case['status'] == "failed"]
    revised_code = revised_backend_code_with_qa_testing_from_llm(state.backend_code, failed_test_cases)
    return {
        f"{code_type}_code": revised_code,
        f"{code_type}_messages": AIMessage(
            content=f"Please review revised {code_type} code and provide additional feedback or type 'Approved' if you're satisfied."
        ),
        f"{code_type}_status": "pending_approval",
        "revised_count": revised_count
    }


## Code Deployment

In [22]:
def code_deployment(state : SDLCState) -> SDLCState:
    print("In code_deployment...")
    return {
        "deployment": "Code has been deployed successfully."
    }


# SDLC Workflow

In [23]:
## Node definition
from langgraph.graph import StateGraph, START, END
from langgraph.checkpoint.memory import MemorySaver
memory = MemorySaver()

sdlc_graph = StateGraph(SDLCState)

# Adding Nodes 
sdlc_graph.add_node("process_project_requirements", process_project_requirements)
sdlc_graph.add_node("generate_user_stories", generate_user_stories)
sdlc_graph.add_node("review_user_stories", review_user_stories)
sdlc_graph.add_node("revised_user_stories", revised_user_stories)

## Functional documents 
sdlc_graph.add_node("create_functional_documents", create_functional_documents)
sdlc_graph.add_node("review_functional_documents", review_functional_documents)
sdlc_graph.add_node("revise_functional_documents", revise_functional_documents)

## Technical documents 
sdlc_graph.add_node("create_technical_documents", create_technical_documents)
sdlc_graph.add_node("review_technical_documents", review_technical_documents)
sdlc_graph.add_node("revise_technical_documents", revise_technical_documents)

## Frontend code 
sdlc_graph.add_node("generate_frontend_code", generate_frontend_code)
sdlc_graph.add_node("review_frontend_code", review_frontend_code)
sdlc_graph.add_node("fix_frontend_code", fix_frontend_code)

## Backend code
sdlc_graph.add_node("generate_backend_code", generate_backend_code)
sdlc_graph.add_node("review_backend_code", review_backend_code)
sdlc_graph.add_node("fix_backend_code", fix_backend_code)

## Security Review
sdlc_graph.add_node("check_code_for_security_issues", check_code_for_security_issues)
sdlc_graph.add_node("security_review", security_review)
sdlc_graph.add_node("fix_code_after_security_review", fix_code_after_security_review)

## Test Cases
sdlc_graph.add_node("write_test_cases", write_test_cases)
sdlc_graph.add_node("test_cases_review", test_cases_review)
sdlc_graph.add_node("fixed_testcases_after_review", fixed_testcases_after_review)

## QA testing
sdlc_graph.add_node("perform_qa_testing", perform_qa_testing)
sdlc_graph.add_node("fixed_code_after_qa_testing", fixed_code_after_qa_testing)

## Code deployment
sdlc_graph.add_node("code_deployment", code_deployment)



## Edges
sdlc_graph.add_edge(START, "process_project_requirements")
sdlc_graph.add_edge("process_project_requirements", "generate_user_stories")
sdlc_graph.add_edge("generate_user_stories", "review_user_stories")
sdlc_graph.add_conditional_edges(
    "review_user_stories",
    should_revise_user_stories,
    {
        "feedback": "revised_user_stories",
        "approved": "create_functional_documents"
    }
)
sdlc_graph.add_edge("revised_user_stories", "review_user_stories")

## Design documents 
sdlc_graph.add_edge("create_functional_documents", "review_functional_documents")
sdlc_graph.add_conditional_edges(
    "review_functional_documents",
    should_revise_function_documents,
    {
        "feedback" : "revise_functional_documents",
        "approved" : "create_technical_documents"
    }
)
sdlc_graph.add_edge("revise_functional_documents", "review_functional_documents")
sdlc_graph.add_edge("create_technical_documents", "review_technical_documents")
sdlc_graph.add_conditional_edges(
    "review_technical_documents",
    should_revise_technical_documents,
    {
        "feedback" : "revise_technical_documents",
        "approved" : "generate_frontend_code"
    }
)
sdlc_graph.add_edge("revise_technical_documents", "review_technical_documents")

## Frontend code 
sdlc_graph.add_edge("generate_frontend_code", "review_frontend_code")
sdlc_graph.add_conditional_edges(
    "review_frontend_code",
    should_fix_frontend_code,
    {
        "feedback" : "fix_frontend_code",
        "approved" : "generate_backend_code"
    }
)
sdlc_graph.add_edge("fix_frontend_code", "review_frontend_code")

## Backend code 
sdlc_graph.add_edge("generate_backend_code", "review_backend_code")
sdlc_graph.add_conditional_edges(
    "review_backend_code",
    should_fix_backend_code,
    {
        "feedback" : "fix_backend_code",
        "approved" : "check_code_for_security_issues"
    }
)
sdlc_graph.add_edge("fix_backend_code", "review_backend_code")

## Security Review
sdlc_graph.add_edge("check_code_for_security_issues", "security_review")
sdlc_graph.add_conditional_edges(
    "security_review",
    should_fix_code_after_security_review,
    {
        "feedback" : "fix_code_after_security_review",
        "approved" : "write_test_cases"
    }
)

sdlc_graph.add_edge("fix_code_after_security_review", "review_backend_code")

## test cases
sdlc_graph.add_edge("write_test_cases", "test_cases_review")
sdlc_graph.add_conditional_edges(
    "test_cases_review",
    should_fix_test_cases,
    {
        "feedback" : "fixed_testcases_after_review",
        "approved" : "perform_qa_testing"
    }
)

sdlc_graph.add_edge("fixed_testcases_after_review", "test_cases_review")

## QA testing
sdlc_graph.add_conditional_edges(
    "perform_qa_testing",
    should_fix_after_qa_testing,
    {
        "failed" : "fixed_code_after_qa_testing",
        "passed" : "code_deployment"
    }
)

sdlc_graph.add_edge("fixed_code_after_qa_testing", "review_backend_code")

sdlc_graph.add_edge("code_deployment", END)

# Compile the graph
sdlc_workflow = sdlc_graph.compile(checkpointer=memory, interrupt_before=["review_user_stories", "review_functional_documents", "review_technical_documents", "review_frontend_code", "review_backend_code", "security_review", "test_cases_review"])

In [24]:
# from IPython.display import Image, display
# display(Image(sdlc_workflow.get_graph().draw_mermaid_png()))

In [25]:
# # from IPython.display import Image, display
# # display(Image(sdlc_workflow.get_graph().draw_mermaid_png()))

# png_data = sdlc_workflow.get_graph().draw_mermaid_png()

# # Save it to a file
# with open("sdlc_workflow.png", "wb") as f:
#     f.write(png_data)

# User stories

In [26]:
project_title = "PayMate: Your Ultimate Payment Companion"
project_description = "PayMate is a comprehensive payment application that allows users to perform seamless transactions using the Unified Payments Interface (UPI). Beyond basic payments, PayMate offers features such as quick loans, bill payments, and a user-friendly interface, making it a one-stop solution for all financial needs."
requirements = ["Implement multi-factor authentication, including biometrics (fingerprint and facial recognition) and MPIN, to secure user accounts.​", "Enable users to link multiple bank accounts and perform instant fund transfers using UPI.", "Provide users with access to instant micro-loans with minimal documentation.", "Allow users to pay utility bills such as electricity, water, gas, and broadband directly through the app."]
requirements_string = "\n".join(requirements)

In [27]:
project_requirements = ProjectRequirements(
    title = project_title,
    description = project_description,
    requirements = requirements
)

initial_story_state = {
    "project_requirements" : project_requirements,
    "user_stories": [],
    "user_story_messages": HumanMessage(content=f"{project_requirements}"),
    "user_story_status": "in_progress",
    "revised_count" : 0,
}

# Thread
thread = {"configurable": {"thread_id": "12345"}}

state = None
for event in sdlc_workflow.stream(initial_story_state, thread, stream_mode="values"):
    state = event

state

In process_project_requirements...
In generate_user_stories...


{'project_requirements': ProjectRequirements(title='PayMate: Your Ultimate Payment Companion', description='PayMate is a comprehensive payment application that allows users to perform seamless transactions using the Unified Payments Interface (UPI). Beyond basic payments, PayMate offers features such as quick loans, bill payments, and a user-friendly interface, making it a one-stop solution for all financial needs.', requirements=['Implement multi-factor authentication, including biometrics (fingerprint and facial recognition) and MPIN, to secure user accounts.\u200b', 'Enable users to link multiple bank accounts and perform instant fund transfers using UPI.', 'Provide users with access to instant micro-loans with minimal documentation.', 'Allow users to pay utility bills such as electricity, water, gas, and broadband directly through the app.']),
 'revised_count': 0,
 'user_stories': [{'story_id': 'US-001',
   'title': 'Secure Account with Multi-Factor Authentication',
   'description

In [28]:
current_state = sdlc_workflow.get_state(thread) 
current_state.next

('review_user_stories',)

In [29]:
user_feedback = "Add user story to buy insurance from the app"
sdlc_workflow.update_state(thread, { "user_story_messages" : HumanMessage(content=f'{user_feedback}')})

for event in sdlc_workflow.stream(None, thread, stream_mode="values"):
    state = event

state

In review_user_stories...
user_review :  add user story to buy insurance from the app
In revised_user_stories...
revised_count :  1


{'project_requirements': ProjectRequirements(title='PayMate: Your Ultimate Payment Companion', description='PayMate is a comprehensive payment application that allows users to perform seamless transactions using the Unified Payments Interface (UPI). Beyond basic payments, PayMate offers features such as quick loans, bill payments, and a user-friendly interface, making it a one-stop solution for all financial needs.', requirements=['Implement multi-factor authentication, including biometrics (fingerprint and facial recognition) and MPIN, to secure user accounts.\u200b', 'Enable users to link multiple bank accounts and perform instant fund transfers using UPI.', 'Provide users with access to instant micro-loans with minimal documentation.', 'Allow users to pay utility bills such as electricity, water, gas, and broadband directly through the app.']),
 'revised_count': 1,
 'user_stories': [{'story_id': 'US-001',
   'title': 'Secure Account with Multi-Factor Authentication',
   'description

In [30]:
user_feedback = "Add user story to buy insurance from the app"
sdlc_workflow.update_state(thread, { "user_story_messages" : HumanMessage(content='Approved'), "revised_count" : 0})

for event in sdlc_workflow.stream(None, thread, stream_mode="values"):
    state = event

state

In review_user_stories...
user_review :  approved
In create_functional_documents...


{'project_requirements': ProjectRequirements(title='PayMate: Your Ultimate Payment Companion', description='PayMate is a comprehensive payment application that allows users to perform seamless transactions using the Unified Payments Interface (UPI). Beyond basic payments, PayMate offers features such as quick loans, bill payments, and a user-friendly interface, making it a one-stop solution for all financial needs.', requirements=['Implement multi-factor authentication, including biometrics (fingerprint and facial recognition) and MPIN, to secure user accounts.\u200b', 'Enable users to link multiple bank accounts and perform instant fund transfers using UPI.', 'Provide users with access to instant micro-loans with minimal documentation.', 'Allow users to pay utility bills such as electricity, water, gas, and broadband directly through the app.']),
 'revised_count': 0,
 'user_stories': [{'story_id': 'US-001',
   'title': 'Secure Account with Multi-Factor Authentication',
   'description

# Documentation

In [31]:
current_state = sdlc_workflow.get_state(thread) 
current_state.next

('review_functional_documents',)

In [32]:
user_feedback = "Revised the document for buy insurance from the app"
sdlc_workflow.update_state(thread, { "functional_messages" : HumanMessage(content=f'{user_feedback}')})
# sdlc_workflow.update_state(thread, { "functional_messages" : HumanMessage(content='Approved'), "revised_count" : 0})

for event in sdlc_workflow.stream(None, thread, stream_mode="values"):
    state = event

state

In review_functional_documents...
user feedback: revised the document for buy insurance from the app
In revise_functional_documents...
user feedback: revised the document for buy insurance from the app
revised_count : 1


{'project_requirements': ProjectRequirements(title='PayMate: Your Ultimate Payment Companion', description='PayMate is a comprehensive payment application that allows users to perform seamless transactions using the Unified Payments Interface (UPI). Beyond basic payments, PayMate offers features such as quick loans, bill payments, and a user-friendly interface, making it a one-stop solution for all financial needs.', requirements=['Implement multi-factor authentication, including biometrics (fingerprint and facial recognition) and MPIN, to secure user accounts.\u200b', 'Enable users to link multiple bank accounts and perform instant fund transfers using UPI.', 'Provide users with access to instant micro-loans with minimal documentation.', 'Allow users to pay utility bills such as electricity, water, gas, and broadband directly through the app.']),
 'revised_count': 1,
 'user_stories': [{'story_id': 'US-001',
   'title': 'Secure Account with Multi-Factor Authentication',
   'description

In [33]:
user_feedback = "Revised the document for buy insurance from the app"
sdlc_workflow.update_state(thread, { "functional_messages" : HumanMessage(content='Approved'), "revised_count" : 0})

for event in sdlc_workflow.stream(None, thread, stream_mode="values"):
    state = event

state

In review_functional_documents...
user feedback: approved
In create_technical_documents...


{'project_requirements': ProjectRequirements(title='PayMate: Your Ultimate Payment Companion', description='PayMate is a comprehensive payment application that allows users to perform seamless transactions using the Unified Payments Interface (UPI). Beyond basic payments, PayMate offers features such as quick loans, bill payments, and a user-friendly interface, making it a one-stop solution for all financial needs.', requirements=['Implement multi-factor authentication, including biometrics (fingerprint and facial recognition) and MPIN, to secure user accounts.\u200b', 'Enable users to link multiple bank accounts and perform instant fund transfers using UPI.', 'Provide users with access to instant micro-loans with minimal documentation.', 'Allow users to pay utility bills such as electricity, water, gas, and broadband directly through the app.']),
 'revised_count': 0,
 'user_stories': [{'story_id': 'US-001',
   'title': 'Secure Account with Multi-Factor Authentication',
   'description

In [34]:
## technical document
current_state = sdlc_workflow.get_state(thread) 
current_state.next

('review_technical_documents',)

In [35]:
# user_feedback = "Revised the technical document for buy insurance from the app"
# # sdlc_workflow.update_state(thread, { "technical_messages" : HumanMessage(content=f'{user_feedback}')})
# # sdlc_workflow.update_state(thread, { "technical_messages" : HumanMessage(content='Approved'), "revised_count" : 0})

# for event in sdlc_workflow.stream(None, thread, stream_mode="values"):
#     state = event
    
# state


In [36]:
user_feedback = "Revised the technical document for buy insurance from the app"
# sdlc_workflow.update_state(thread, { "technical_messages" : HumanMessage(content=f'{user_feedback}')})
sdlc_workflow.update_state(thread, { "technical_messages" : HumanMessage(content='Approved'), "revised_count" : 0})

for event in sdlc_workflow.stream(None, thread, stream_mode="values"):
    state = event
    
state


In review_technical_documents...


user feedback: approved
In generate_frontend_code...


{'project_requirements': ProjectRequirements(title='PayMate: Your Ultimate Payment Companion', description='PayMate is a comprehensive payment application that allows users to perform seamless transactions using the Unified Payments Interface (UPI). Beyond basic payments, PayMate offers features such as quick loans, bill payments, and a user-friendly interface, making it a one-stop solution for all financial needs.', requirements=['Implement multi-factor authentication, including biometrics (fingerprint and facial recognition) and MPIN, to secure user accounts.\u200b', 'Enable users to link multiple bank accounts and perform instant fund transfers using UPI.', 'Provide users with access to instant micro-loans with minimal documentation.', 'Allow users to pay utility bills such as electricity, water, gas, and broadband directly through the app.']),
 'revised_count': 0,
 'user_stories': [{'story_id': 'US-001',
   'title': 'Secure Account with Multi-Factor Authentication',
   'description

### Frontend Code

In [37]:
## technical document
current_state = sdlc_workflow.get_state(thread) 
current_state.next

('review_frontend_code',)

In [38]:
user_feedback = "Revised the frontend code for buy insurance from the app"
sdlc_workflow.update_state(thread, { "frontend_messages" : HumanMessage(content=f'{user_feedback}')})
# sdlc_workflow.update_state(thread, { "frontend_messages" : HumanMessage(content='Approved'), "revised_count" : 0})

for event in sdlc_workflow.stream(None, thread, stream_mode="values"):
    state = event
    
state


In review_frontend_code
user feedback: revised the frontend code for buy insurance from the app
In fix_frontend_code...
revised_count : 1


{'project_requirements': ProjectRequirements(title='PayMate: Your Ultimate Payment Companion', description='PayMate is a comprehensive payment application that allows users to perform seamless transactions using the Unified Payments Interface (UPI). Beyond basic payments, PayMate offers features such as quick loans, bill payments, and a user-friendly interface, making it a one-stop solution for all financial needs.', requirements=['Implement multi-factor authentication, including biometrics (fingerprint and facial recognition) and MPIN, to secure user accounts.\u200b', 'Enable users to link multiple bank accounts and perform instant fund transfers using UPI.', 'Provide users with access to instant micro-loans with minimal documentation.', 'Allow users to pay utility bills such as electricity, water, gas, and broadband directly through the app.']),
 'revised_count': 1,
 'user_stories': [{'story_id': 'US-001',
   'title': 'Secure Account with Multi-Factor Authentication',
   'description

In [39]:
user_feedback = "Revised the frontend code for buy insurance from the app"
# sdlc_workflow.update_state(thread, { "frontend_messages" : HumanMessage(content=f'{user_feedback}')})
sdlc_workflow.update_state(thread, { "frontend_messages" : HumanMessage(content='Approved'),  "revised_count" : 0})

for event in sdlc_workflow.stream(None, thread, stream_mode="values"):
    state = event
    
state


In review_frontend_code
user feedback: approved
In generate_backend_code...


{'project_requirements': ProjectRequirements(title='PayMate: Your Ultimate Payment Companion', description='PayMate is a comprehensive payment application that allows users to perform seamless transactions using the Unified Payments Interface (UPI). Beyond basic payments, PayMate offers features such as quick loans, bill payments, and a user-friendly interface, making it a one-stop solution for all financial needs.', requirements=['Implement multi-factor authentication, including biometrics (fingerprint and facial recognition) and MPIN, to secure user accounts.\u200b', 'Enable users to link multiple bank accounts and perform instant fund transfers using UPI.', 'Provide users with access to instant micro-loans with minimal documentation.', 'Allow users to pay utility bills such as electricity, water, gas, and broadband directly through the app.']),
 'revised_count': 0,
 'user_stories': [{'story_id': 'US-001',
   'title': 'Secure Account with Multi-Factor Authentication',
   'description

### Backend code

In [40]:
## technical document
current_state = sdlc_workflow.get_state(thread) 
current_state.next

('review_backend_code',)

In [41]:
user_feedback = "Revised the backend code for buy insurance from the app"
sdlc_workflow.update_state(thread, { "backend_messages" : HumanMessage(content=f'{user_feedback}')})
# sdlc_workflow.update_state(thread, { "backend_messages" : HumanMessage(content='Approved'), "revised_count" : 0})

for event in sdlc_workflow.stream(None, thread, stream_mode="values"):
    state = event
    
state


In review_backend_code
user feedback: revised the backend code for buy insurance from the app
In fix_backend_code...
revised_count : 1


{'project_requirements': ProjectRequirements(title='PayMate: Your Ultimate Payment Companion', description='PayMate is a comprehensive payment application that allows users to perform seamless transactions using the Unified Payments Interface (UPI). Beyond basic payments, PayMate offers features such as quick loans, bill payments, and a user-friendly interface, making it a one-stop solution for all financial needs.', requirements=['Implement multi-factor authentication, including biometrics (fingerprint and facial recognition) and MPIN, to secure user accounts.\u200b', 'Enable users to link multiple bank accounts and perform instant fund transfers using UPI.', 'Provide users with access to instant micro-loans with minimal documentation.', 'Allow users to pay utility bills such as electricity, water, gas, and broadband directly through the app.']),
 'revised_count': 1,
 'user_stories': [{'story_id': 'US-001',
   'title': 'Secure Account with Multi-Factor Authentication',
   'description

In [42]:
user_feedback = "Revised the backend code for buy insurance from the app"
# sdlc_workflow.update_state(thread, { "backend_messages" : HumanMessage(content=f'{user_feedback}')})
sdlc_workflow.update_state(thread, { "backend_messages" : HumanMessage(content='Approved'), "revised_count" : 0})

for event in sdlc_workflow.stream(None, thread, stream_mode="values"):
    state = event
    
state


In review_backend_code
user feedback: approved
In check_code_for_security_issues...


{'project_requirements': ProjectRequirements(title='PayMate: Your Ultimate Payment Companion', description='PayMate is a comprehensive payment application that allows users to perform seamless transactions using the Unified Payments Interface (UPI). Beyond basic payments, PayMate offers features such as quick loans, bill payments, and a user-friendly interface, making it a one-stop solution for all financial needs.', requirements=['Implement multi-factor authentication, including biometrics (fingerprint and facial recognition) and MPIN, to secure user accounts.\u200b', 'Enable users to link multiple bank accounts and perform instant fund transfers using UPI.', 'Provide users with access to instant micro-loans with minimal documentation.', 'Allow users to pay utility bills such as electricity, water, gas, and broadband directly through the app.']),
 'revised_count': 0,
 'user_stories': [{'story_id': 'US-001',
   'title': 'Secure Account with Multi-Factor Authentication',
   'description

## Security Review

In [43]:
## technical document
current_state = sdlc_workflow.get_state(thread) 
current_state.next

('security_review',)

In [44]:
user_feedback = "Revised the security review for buy insurance from the app"
# sdlc_workflow.update_state(thread, { "security_reviews_messages" : HumanMessage(content=f'{user_feedback}')})
sdlc_workflow.update_state(thread, { "security_reviews_messages" : HumanMessage(content='Approved'), "revised_count" : 0})

for event in sdlc_workflow.stream(None, thread, stream_mode="values"):
    state = event
    
state


In security_review...
user feedback: approved
In write_test_cases...


{'project_requirements': ProjectRequirements(title='PayMate: Your Ultimate Payment Companion', description='PayMate is a comprehensive payment application that allows users to perform seamless transactions using the Unified Payments Interface (UPI). Beyond basic payments, PayMate offers features such as quick loans, bill payments, and a user-friendly interface, making it a one-stop solution for all financial needs.', requirements=['Implement multi-factor authentication, including biometrics (fingerprint and facial recognition) and MPIN, to secure user accounts.\u200b', 'Enable users to link multiple bank accounts and perform instant fund transfers using UPI.', 'Provide users with access to instant micro-loans with minimal documentation.', 'Allow users to pay utility bills such as electricity, water, gas, and broadband directly through the app.']),
 'revised_count': 0,
 'user_stories': [{'story_id': 'US-001',
   'title': 'Secure Account with Multi-Factor Authentication',
   'description

## Test Cases

In [45]:
## technical document
current_state = sdlc_workflow.get_state(thread) 
current_state.next

('test_cases_review',)

In [46]:
user_feedback = "Revised the test cases for buy insurance from the app"
sdlc_workflow.update_state(thread, { "test_cases_messages" : HumanMessage(content=f'{user_feedback}')})
# sdlc_workflow.update_state(thread, { "test_cases_messages" : HumanMessage(content='Approved'), "revised_count" : 0})

for event in sdlc_workflow.stream(None, thread, stream_mode="values"):
    state = event
    
state


In test_cases_review...
user feedback: revised the test cases for buy insurance from the app
In fixed_testcases_after_review...
revised_count : 1


{'project_requirements': ProjectRequirements(title='PayMate: Your Ultimate Payment Companion', description='PayMate is a comprehensive payment application that allows users to perform seamless transactions using the Unified Payments Interface (UPI). Beyond basic payments, PayMate offers features such as quick loans, bill payments, and a user-friendly interface, making it a one-stop solution for all financial needs.', requirements=['Implement multi-factor authentication, including biometrics (fingerprint and facial recognition) and MPIN, to secure user accounts.\u200b', 'Enable users to link multiple bank accounts and perform instant fund transfers using UPI.', 'Provide users with access to instant micro-loans with minimal documentation.', 'Allow users to pay utility bills such as electricity, water, gas, and broadband directly through the app.']),
 'revised_count': 1,
 'user_stories': [{'story_id': 'US-001',
   'title': 'Secure Account with Multi-Factor Authentication',
   'description

In [47]:
## technical document
current_state = sdlc_workflow.get_state(thread) 
current_state.next

('test_cases_review',)

In [48]:
user_feedback = "Revised the test cases for buy insurance from the app"
# sdlc_workflow.update_state(thread, { "test_cases_messages" : HumanMessage(content=f'{user_feedback}')})
sdlc_workflow.update_state(thread, { "test_cases_messages" : HumanMessage(content='Approved'), "revised_count" : 0})

for event in sdlc_workflow.stream(None, thread, stream_mode="values"):
    state = event
    
state


In test_cases_review...
user feedback: approved
In perform_qa_testing...
In code_deployment...


{'project_requirements': ProjectRequirements(title='PayMate: Your Ultimate Payment Companion', description='PayMate is a comprehensive payment application that allows users to perform seamless transactions using the Unified Payments Interface (UPI). Beyond basic payments, PayMate offers features such as quick loans, bill payments, and a user-friendly interface, making it a one-stop solution for all financial needs.', requirements=['Implement multi-factor authentication, including biometrics (fingerprint and facial recognition) and MPIN, to secure user accounts.\u200b', 'Enable users to link multiple bank accounts and perform instant fund transfers using UPI.', 'Provide users with access to instant micro-loans with minimal documentation.', 'Allow users to pay utility bills such as electricity, water, gas, and broadband directly through the app.']),
 'revised_count': 0,
 'user_stories': [{'story_id': 'US-001',
   'title': 'Secure Account with Multi-Factor Authentication',
   'description

In [49]:
## technical document
current_state = sdlc_workflow.get_state(thread) 
current_state.next

()

## QA testing

In [50]:
## technical document
current_state = sdlc_workflow.get_state(thread) 
current_state.next

()

In [51]:
## technical document
current_state = sdlc_workflow.get_state(thread) 
current_state.next

()